In [27]:
import pandas as pd
from goessolarretriever import Retriever, Satellite, Product
from glob import glob
from datetime import datetime
import numpy as np
from astropy.io import fits
import os
import matplotlib.pyplot as plt
%matplotlib notebook

First we specify the satellite, product, and time interval to retrive. We can search for all the data in that time interval. 

*Note: the current time interval will take a while. Decrease the window to save time.*

In [65]:
satellite = Satellite.GOES16
product = Product.suvi_l2_ci195

start = datetime(2019, 2, 1)
end = datetime(2020, 4, 8)

r = Retriever()
df = r.search(satellite, product, start, end)

Now, we can inspect the table and see how many entries there are. 

In [72]:
df

,file_name,date_begin,date_end,date_edited,file_size,url
0,dr_suvi-l2-ci195_g16_s20190201T000000Z_e201902...,2019-02-01 00:00:00,2019-02-01 00:04:00,2019-02-01 00:10:00,1.4M,https://data.ngdc.noaa.gov/platforms/solar-spa...
1,dr_suvi-l2-ci195_g16_s20190201T000400Z_e201902...,2019-02-01 00:04:00,2019-02-01 00:08:00,2019-02-01 00:14:00,1.4M,https://data.ngdc.noaa.gov/platforms/solar-spa...
2,dr_suvi-l2-ci195_g16_s20190201T000800Z_e201902...,2019-02-01 00:08:00,2019-02-01 00:12:00,2019-02-01 00:18:00,1.4M,https://data.ngdc.noaa.gov/platforms/solar-spa...
3,dr_suvi-l2-ci195_g16_s20190201T001200Z_e201902...,2019-02-01 00:12:00,2019-02-01 00:16:00,2019-02-01 00:22:00,1.4M,https://data.ngdc.noaa.gov/platforms/solar-spa...
4,dr_suvi-l2-ci195_g16_s20190201T001600Z_e201902...,2019-02-01 00:16:00,2019-02-01 00:20:00,2019-02-01 00:26:00,1.4M,https://data.ngdc.noaa.gov/platforms/solar-spa...
...,...,...,...,...,...,...
114192,dr_suvi-l2-ci195_g16_s20200408T175200Z_e202004...,2020-04-08 17:52:00,2020-04-08 17:56:00,2020-04-08 18:01:00,1.3M,https://data.ngdc.noaa.gov/platforms/solar-spa...
114193,dr_suvi-l2-ci195_g16_s20200408T203600Z_e202004...,2020-04-08 20:36:00,2020-04-08 20:40:00,2020-04-08 20:45:00,5.6K,https://data.ngdc.noaa.gov/platforms/solar-spa...
114194,dr_suvi-l2-ci195_g16_s20200408T204000Z_e202004...,2020-04-08 20:40:00,2020-04-08 20:44:00,2020-04-08 20:49:00,5.6K,https://data.ngdc.noaa.gov/platforms/solar-spa...
114195,dr_suvi-l2-ci195_g16_s20200408T204400Z_e202004...,2020-04-08 20:44:00,2020-04-08 20:48:00,2020-04-08 20:53:00,5.6K,https://data.ngdc.noaa.gov/platforms/solar-spa...


We can resample to get one image for every 4 hours and then examine the list. 

In [67]:
times = pd.DataFrame(list(zip(df['file_name'].array, df['url'].array)), index=df['date_begin'], columns=['file_name', 'url'])

In [68]:
times = times.resample("4H").asfreq().replace(np.nan, '', regex=True)

In [69]:
times

,file_name,url
date_begin,,
2019-02-01 00:00:00,dr_suvi-l2-ci195_g16_s20190201T000000Z_e201902...,https://data.ngdc.noaa.gov/platforms/solar-spa...
2019-02-01 04:00:00,dr_suvi-l2-ci195_g16_s20190201T040000Z_e201902...,https://data.ngdc.noaa.gov/platforms/solar-spa...
2019-02-01 08:00:00,dr_suvi-l2-ci195_g16_s20190201T080000Z_e201902...,https://data.ngdc.noaa.gov/platforms/solar-spa...
2019-02-01 12:00:00,dr_suvi-l2-ci195_g16_s20190201T120000Z_e201902...,https://data.ngdc.noaa.gov/platforms/solar-spa...
2019-02-01 16:00:00,dr_suvi-l2-ci195_g16_s20190201T160000Z_e201902...,https://data.ngdc.noaa.gov/platforms/solar-spa...
...,...,...
2020-04-08 04:00:00,dr_suvi-l2-ci195_g16_s20200408T040000Z_e202004...,https://data.ngdc.noaa.gov/platforms/solar-spa...
2020-04-08 08:00:00,dr_suvi-l2-ci195_g16_s20200408T080000Z_e202004...,https://data.ngdc.noaa.gov/platforms/solar-spa...
2020-04-08 12:00:00,dr_suvi-l2-ci195_g16_s20200408T120000Z_e202004...,https://data.ngdc.noaa.gov/platforms/solar-spa...


Finally, we retrieve all the images requested and save them in the specified folder. 

In [70]:
r.retrieve(times, "/home/marcus/Desktop/suvi_movie/data/")

2598it [35:50,  1.21it/s]


In [71]:
times.to_csv("/home/marcus/Desktop/suvi_movie/times.csv")

To examine our results, we can make a movie of the images. 

In [ ]:
def my_scale(img, power=0.25):
    return np.sign(img) * np.power(np.abs(img), power)

i = 0
for date, row in times.iterrows():
    if row['url']: 
        try:
            img = fits.open(os.path.join("/home/marcus/Desktop/suvi_movie/data/", row['file_name']))
            data = my_scale(img[1].data)
        except:
            data = np.zeros((1280, 1280))
    else:
        data = np.zeros((1280,1280))
    date_str = datetime.strftime(date, "%Y-%m-%d %H:%M")
    fig, ax = plt.subplots()
    ax.imshow(data, vmin=0, vmax=1.2, origin='lower')
    ax.set_axis_off()
    ax.set_title(date_str)
    fig.savefig("/home/marcus/Desktop/suvi_movie/pngs/image{:04d}.png".format(i))
    plt.close()
    i+=1

Run the following to make movies:

-```ffmpeg -framerate 10 -i image%04d.png -s:v 1280x720 -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p ../slow.mp4```

-```ffmpeg -framerate 60 -i image%04d.png -s:v 1280x720 -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p ../fast.mp4```